In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize

pd.set_option('display.max_colwidth',1000)

In [2]:
col_list = ["ID", "FULL TEXT","CREATED AT"]
data = pd.read_csv('get_tweets.csv', usecols=col_list)
data.columns = ['ID','FULL_TEXT','CREATED_AT']
print(data)

                      ID  \
0    1414162517924843525   
1    1414150972830535687   
2    1414150959186522115   
3    1414112167616339970   
4    1414105155205693442   
..                   ...   
391  1412692492667981827   
392  1412321237531987972   
393  1411720788755959808   
394  1411580990313484288   
395  1411541693770321928   

                                                                                                                                                                                                                                                                                                                                                        FULL_TEXT  \
0                                                                                          අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) 

In [3]:
#stopwords = ['අන්තර්ජාලය','ටික','නර්ස්','බාලවයස්කාර']
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    #tokens = [word for word in tokens if word not in stopwords]
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [4]:
data['cleaned_msg'] = data['FULL_TEXT'].apply(lambda x: clean_text(x))
data.head()

,ID,FULL_TEXT,CREATED_AT,cleaned_msg
0,1414162517924843525,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,2021-07-11 09:59:52,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද අත්අඩංගුවට ගනු ලැබ තිබේ
1,1414150972830535687,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,2021-07-11 09:14:00,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන ක්
2,1414150959186522115,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,2021-07-11 09:13:56,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි
3,1414112167616339970,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",2021-07-11 06:39:48,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල මෙහි ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් එම වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත
4,1414105155205693442,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,2021-07-11 06:11:56,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ සුනිල් ගුණවර්ධන


In [5]:
#######################c Remove Stop words Method 1: Remove stop words  #########################################

In [6]:
cleaned_data = data['cleaned_msg']
wli = []
for l in cleaned_data:
    l = l.split()
    
    for w in l:
        wli.append(w)


In [7]:
# create word dictionary word and word count
wli.sort()
wd = {} 
for w in wli:
    wd[w] = wli.count(w)
    
print('\n{:^8}{:^8}'.format('word','count'))
for w in wd:
    print('{:^8}{:^8}'.format(w,wd[w]))
    
print("")
    


  word   count  
   අ       1    
  අංශ      8    
 අංශත්     1    
 අංශයට     1    
 අංශයේ     2    
 අක්කේ     1    
 අක්රීය    1    
 අගමැති    5    
අගමැතිගෙ   1    
අගමැතිත්   1    
 අඟවයි     1    
  අඟවා     2    
 අජිත්     2    
අටපට්ටමට   2    
  අඩවි     1    
  අඩු      11   
අඩුනිසා    1    
  අඩුම     1    
 අඩුයි     2    
අඩුවීමට    1    
අඩුවෙන්    1    
අඩුවෙලා    1    
 අඬනවා     2    
අඬවැඩියාකාරයෝ   1    
 අතකොළු    1    
  අතර      9    
 අතරින්    2    
අතහැරලා    1    
  අත්      2    
අත්අඩංගුවට   2    
අත්තනෝමතික   1    
 අත්හදා    1    
 අත්හැර    1    
අත්හැරදමාතිබූදිවුලපිටියදිස්ත්   1    
අත්හිටුවයි   1    
  අති      1    
 අතිරේක    1    
 අතිශය     2    
  අතීත     1    
   අද      29   
අදක්ෂයෙක්ට   1    
 අදකාලේ    1    
  අදට      1    
 අදටත්     1    
 අදදින     1    
  අදහස     2    
 අදහස්     2    
  අදාල     4    
අධ්යක්ෂ    5    
අධ්යක්ෂක   3    
අධ්යක්ෂණය   1    
අධ්යාපන    2    
අධ්යාපනය   3    
අධ්යාපනයයි   1    
අධිමාත්රික   1    
අධිවේගී 

  නායක     1    
 නායකයා    1    
නායකයින්   1    
නායකයෙක්   1    
 නායකයො    1    
   නැ      2    
නැග්ගුවා   1    
  නැගී     2    
 නැගුම     1    
නැගෙනවා    1    
  නැත      1    
 නැත්තෙ    2    
 නැත්තේ    1    
නැත්නම්    3    
  නැති     11   
නැතිඅඩපණසිහිසුන්   1    
නැතිකොට    1    
නැතිනම්    1    
 නැතිව     1    
 නැතුං     1    
 නැතුව     3    
  නැද      1    
 නැද්ද     9    
  නැවත     2    
 නැවතත්    1    
  නැහැ     8    
 නැහැනෙ    1    
   නෑ      32   
නෑකොරෝනා   1    
  නෑනෙ     1    
  නෑනේ     3    
නෑල්ලුද    1    
 නිකන්     1    
 නිකමට     1    
 නිකම්     1    
 නිකුත්    4    
නිට්ටාවටම   1    
  නිතර     2    
  නිති     1    
නිතියක්    1    
 නිදහස්    7    
නින්දිත    1    
 නිපදවන    1    
 නිමාලි    1    
 නියමිත    3    
නියමිතය    1    
නියමිතව    1    
 නියැලී    2    
නියෝජනය    3    
නිර්දේශය   1    
නිර්මාණකරුවන්ට   1    
නිරිතදිග   2    
නිරූපණය    1    
නිරෝධායන   2    
නිරෝධායනය   2    
නිරෝධායනයට   1    
නිලධාරියාට   1    
නිලධාරියෙක්

 වසංගත     3    
 වසංගතට    1    
 වසංගතය    7    
වසංගතයක්   1    
වසංගතයක්වත්   1    
වසංගතයේ    2    
  වසර      3    
 වසරේදී    1    
  වසා      1    
වසාදැමීමෙන්   1    
 වහංගත     1    
 වහගෙන     1    
 වහන්න     1    
වහන්සේට    1    
  වහයි     1    
  වහලා     1    
ව්යවසනයෙන්   1    
ව්යවස්ථාව   1    
ව්යවස්ථාවට   1    
ව්යවස්ථාවේ   1    
ව්යවසාය    1    
 ව්යාපර    1    
ව්යාප්ත    3    
ව්යාප්තිය   4    
ව්යාපාර    1    
 වාදනය     1    
 වාර්තා    42   
වාර්තාගත   1    
වාර්තාව    4    
වාර්තාවක්   2    
වාර්තාවන   1    
වාර්තාවලට   1    
වාර්තාවීමේ   2    
වාර්තාවෙයි   1    
වාර්තාවේ   5    
වාර්ථාගත   1    
වාර්ෂික    1    
  වාරැ     1    
වාසනාවට    1    
 වාහනේ     1    
වැක්සින්   1    
වැට්ටවීමයි   1    
 වැටිලා    1    
 වැටුණු    1    
වැටෙන්න    1    
  වැඩ      10   
 වැඩඇයි    1    
  වැඩක     1    
 වැඩක්     2    
  වැඩට     1    
 වැඩත්     1    
වැඩපිළිවෙලක්   1    
වැඩමුලුව   1    
වැඩවර්ජනය   1    
වැඩවර්ජනයෙ   1    
වැඩසටහන    2    
වැඩසටහනක්   2    
 

In [8]:
stopword = []
for w in wd:
    if(wd[w]>10):
        stopword.append(w)
print(stopword)

['අඩු', 'අද', 'අපි', 'අපේ', 'අවුරුදු', 'ආසාදිත', 'ආසාදිතයින්', 'ඇත', 'ඇති', 'ඇමති', 'ඊයේ', 'එක', 'එකට', 'එන්නත', 'එම', 'එහෙම', 'ඒ', 'ඔය', 'ඔලිම්පික්', 'කට', 'කර', 'කරන', 'කරන්න', 'කරලා', 'ක්', 'ක්රිකට්', 'ක්රීඩා', 'කාන්තා', 'කියලා', 'කොරෝනා', 'ගැන', 'ගැනේ', 'තමයි', 'තවත්', 'ද', 'දැන්', 'දෙනෙකු', 'නම්', 'නැති', 'නෑ', 'නිසා', 'ප්රදේශ', 'පොකුර', 'බව', 'බැසිල්', 'මරණ', 'මුළු', 'මෙරට', 'මෙරටින්', 'මේ', 'රට', 'රටේ', 'රෝගීන්', 'ලංකා', 'වගේ', 'වන', 'වල', 'වලට', 'වාර්තා', 'වුණු', 'වෙන්න', 'වෙයි', 'වෙලා', 'ශ්රී', 'සංඛ්යාව', 'සඳහා', 'සහ', 'සිට', 'සුවය', 'සෞඛ්ය', 'සෞඛ්යය', 'හඳුනා', 'හරි', 'හා', 'හේතුවෙන්']


In [9]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = " ".join([word for word in tokens if word not in stopword])
    return tokens

In [10]:
data['remove_stopwords'] = data['cleaned_msg'].apply(lambda x: clean_text(x))
data.head()

,ID,FULL_TEXT,CREATED_AT,cleaned_msg,remove_stopwords
0,1414162517924843525,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,2021-07-11 09:59:52,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද අත්අඩංගුවට ගනු ලැබ තිබේ,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා කාර්යාංශය විසින් අත්අඩංගුවට ගනු ලැබ තිබේ
1,1414150972830535687,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,2021-07-11 09:14:00,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන ක්,අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන
2,1414150959186522115,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,2021-07-11 09:13:56,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි,මේසපන්දුව දියුණු කරන්නට අමාත්යංශය ගිවිසුම්ගත
3,1414112167616339970,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",2021-07-11 06:39:48,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල මෙහි ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් එම වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල සමාජ පිල මෙහි ලංකව විවිධ අවධි නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් වසරේදී සමාජයට
4,1414105155205693442,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,2021-07-11 06:11:56,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ සුනිල් ගුණවර්ධන,ඇමැතිතුමා තවමත් අය තැන්වලට දාගෙන සුනිල් ගුණවර්ධන


In [11]:
############################ Method 2 #############################################################

In [21]:
crimefile = open('StopWords', 'r', encoding='utf-8')
allRows =[]
reader = csv.reader(crimefile)
allRows = row for row in reader

SyntaxError: invalid syntax (<ipython-input-21-1c617eba03ca>, line 4)

In [22]:
print(allRows)

<generator object <genexpr> at 0x0000026D77354A98>
